In [22]:
import numpy as np
import pandas as pd
import altair as alt
import datetime
import requests

Ok so we gotta read the stuff in from my google sheet where I track time.

In [2]:
sheetUrl = 'https://docs.google.com/spreadsheets/d/1Qq13N2dgpU8TWwLLJKsmauKQM1Damvykkn2nOnFQ56A/edit#gid=580980480'
csvFromSheet = '/'.join(sheetUrl.split('/')[:-1] + ['export?format=csv'])
df = pd.read_csv(csvFromSheet, parse_dates=['start', 'end'])
rawData = df[df['start'] != df['end']].sort_values('start')


/Users/matthewcaseres/.local/share/virtualenvs/productivity-dashboard-d7n1JZqq/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname M identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


See? Run the cell below using ctrl/cmd + enter

In [3]:
rawData

,category,activity,start,end,hours,Unnamed: 5
0,night reading,night reading,2021-03-03 18:28:16,2021-03-03 18:34:55,0.110780,NaN
1,wake up,wake up,2021-03-04 02:28:24,2021-03-04 04:09:47,1.689601,NaN
2,work,school,2021-03-04 04:09:47,2021-03-04 05:10:50,1.017760,NaN
3,food,food,2021-03-04 05:10:50,2021-03-04 06:07:20,0.941574,NaN
4,work,school,2021-03-04 06:07:20,2021-03-04 07:08:00,1.011135,NaN
...,...,...,...,...,...,...
686,work,open source,2021-04-10 09:05:05,2021-04-10 09:52:41,0.793220,NaN
687,work,open source,2021-04-10 10:19:39,2021-04-10 10:49:41,0.500527,NaN
688,work,open source,2021-04-10 11:04:51,2021-04-10 11:28:34,0.395420,NaN
689,work,open source,2021-04-10 11:37:06,2021-04-10 11:56:48,0.328313,NaN


Basically take rawData and make graphs in Altair. The graph below was hard to make... maybe try something simpler to start.

In [9]:
min(rawData['start']).floor('D')
max(rawData['end']).floor('D')

Timestamp('2021-03-03 00:00:00')

In [13]:

# Insert NA between things
fillAfter = rawData.loc[rawData['start'].shift(-1) != rawData['end'], 'end']
NAsDict = {'category': ["N/A"]*len(fillAfter), 'activity': ["N/A"]*len(fillAfter), "start": fillAfter, "end": [None]*len(fillAfter)}
NAs = pd.DataFrame(NAsDict)[:-1]

# These are for the beginning and end
beginningEndDict = {
    'category': ["N/A"]*2, 
    'activity': ["N/A"]*2, 
    "start": [min(rawData['start']).floor('D'), max(rawData['end'])], 
    "end": [min(rawData['start']), max(rawData['end']).ceil('D')]
}
beginningEnd = pd.DataFrame(beginningEndDict)

# This should come after everything is partitioned
midnightSeries = pd.date_range(min(rawData['start']).ceil('D'), max(rawData['end']).floor('D'))
midnightsDict = {'activity': [None]*len(midnightSeries), 'start': midnightSeries, 'end': [None]*len(midnightSeries)}
midnights = pd.DataFrame(midnightsDict)

# combine the NA with the non-NA
all = pd.concat([rawData, NAs, beginningEnd, midnights], ignore_index=True)
all = all.sort_values('start')
# Fill NA for added rows with correct values
all['end'] = all['end'].fillna(all['start'].shift(-1))
all['activity'] = all['activity'].fillna(method='ffill')
all['category'] = all['category'].fillna(method='ffill')
# For days crossing over, truncate at midnight
all.loc[all['start'].dt.day != all['end'].dt.day, 'end'] = all['start'].dt.ceil('D')
all.loc[all['start'] == all['end'], 'end'] = all['start'].shift(-1)
all['Hour'] = (all['end'] - all['start']).dt.total_seconds()/3600
all['day'] = all['start'].dt.floor('D')

selection = alt.selection_multi(fields=['activity'])
# multi = alt.selection_multi()
BLOCKED = []
alt.Chart(all).mark_bar().encode(
    x = alt.X('monthdate(start):O', axis=alt.Axis(title='Date')),
    y=alt.Y('Hour', axis=alt.Axis(title="Time"), scale=alt.Scale(domain=(2, 20), clamp=True)), 
    order=alt.Order('start'), 
    color=alt.condition(alt.FieldOneOfPredicate(field='activity', oneOf=['N/A']+BLOCKED), alt.value('white'), alt.Color('activity', legend=None)),
    opacity=alt.condition(selection, alt.value(1), alt.value(.2)),
    tooltip=[
        'activity:N', 
        alt.Tooltip('Hour:Q', format='.2f', title='hours'), 
        alt.Tooltip('start:T', format='%-I:%M %p', title='started'),
        alt.Tooltip('end:T', format='%-I:%M %p', title='ended')]
).add_selection(
    selection
)

alt.Chart(...)

In [142]:
def getRollingAvgs(allData, low, high, WINDOW_SIZE):
    allData = allData.iloc[:, 0:3]
    allData['s_trunc'] = allData['start'].dt.floor('D') + datetime.timedelta(hours=low)
    allData['e_trunc'] = allData['start'].dt.floor('D') + datetime.timedelta(hours=high)
    allData['log'] = (allData[['end', 'e_trunc']].min(axis=1)-all[['start', 's_trunc']].max(axis=1)).dt.total_seconds()
    allData['log'] = np.where(allData['log'] < 0, 0, all['log'])/3600
    allData['day'] = allData['start'].dt.floor('D')
    all_pivot = pd.pivot_table(allData,values='log',index='day',columns=['activity'],aggfunc=np.sum, fill_value=0)
    rolling_avgs = all_pivot.rolling(WINDOW_SIZE, min_periods=1).mean()
    rolling_sums = all_pivot.rolling(WINDOW_SIZE).sum()
    tidy_rolling_avgs = rolling_avgs.reset_index().melt(id_vars=['day'], value_name='mean')
    tidy_rolling_sums = rolling_sums.reset_index().melt(id_vars=['day'], value_name='total')
    tidy_rolling_avgs['total'] = tidy_rolling_sums['total']
    tidy_rolling_avgs['lookback'] = tidy_rolling_sums['day'] + pd.DateOffset(days=-WINDOW_SIZE)
    return tidy_rolling_avgs
# all.groupby('activity').agg(
#     hours=('log', sum)
# ).sort_values('hours', ascending=False)


INCLUDED = ['school']

tidy_rolling_avgs = getRollingAvgs(all, 0, 24, 7)

selection = alt.selection_multi(fields=['activity'])
alt.Chart(tidy_rolling_avgs).mark_bar().encode(
    x = alt.X('monthdate(day):O', axis=alt.Axis(title='Date')),
    y = alt.Y('mean', axis=alt.Axis(title="Hours")), 
    order=alt.Order('mean', sort="descending"), 
    color=alt.Color('activity', legend=None),
    tooltip=[
        'activity:N', 
        alt.Tooltip('mean:Q', format='.2f', title='mean'), 
        alt.Tooltip('total:Q', format='.2f', title='total'),
        alt.Tooltip('monthdate(day):O', title='day'),
        alt.Tooltip('monthdate(lookback):O', title='lookback to')]
).transform_filter(alt.FieldOneOfPredicate(field='activity', oneOf=INCLUDED))



alt.Chart(...)

In [19]:


def getRollingAvgs(ACTIVITY_OR_CATEGORY, allData, low, high, WINDOW_SIZE):
    # allData = allData.iloc[:, 1:4]
    allData = allData.loc[:, [ACTIVITY_OR_CATEGORY, 'start', 'end']]
    allData['s_trunc'] = allData['start'].dt.floor(
        'D') + datetime.timedelta(hours=low)
    allData['e_trunc'] = allData['start'].dt.floor(
        'D') + datetime.timedelta(hours=high)
    allData['log'] = (allData[['end', 'e_trunc']].min(
        axis=1)-allData[['start', 's_trunc']].max(axis=1)).dt.total_seconds()
    allData['log'] = np.where(allData['log'] < 0, 0, allData['log'])/3600
    allData['day'] = allData['start'].dt.floor('D')
    all_pivot = pd.pivot_table(allData, values='log', index='day', columns=[
                               ACTIVITY_OR_CATEGORY], aggfunc=np.sum, fill_value=0)
    rolling_avgs = all_pivot.rolling(WINDOW_SIZE, min_periods=1).mean()
    rolling_sums = all_pivot.rolling(WINDOW_SIZE).sum()
    tidy_rolling_avgs = rolling_avgs.reset_index().melt(
        id_vars=['day'], value_name='mean')
    tidy_rolling_sums = rolling_sums.reset_index().melt(
        id_vars=['day'], value_name='total')
    tidy_rolling_avgs['total'] = tidy_rolling_sums['total']
    tidy_rolling_avgs['lookback'] = tidy_rolling_sums['day'] + pd.DateOffset(days=-WINDOW_SIZE+1)
    return tidy_rolling_avgs

ACTIVITY_OR_CATEGORY="category"
tidy_rolling_avgs = getRollingAvgs(ACTIVITY_OR_CATEGORY, all, 0, 24, 1).sort_values(by="day")

tidy_rolling_avgs


,day,category,mean,total,lookback
0,2021-03-03,N/A,23.889167,23.889167,2021-03-03
624,2021-03-03,work,0.000000,0.000000,2021-03-03
117,2021-03-03,food,0.000000,0.000000,2021-03-03
78,2021-03-03,chores,0.000000,0.000000,2021-03-03
390,2021-03-03,shower,0.000000,0.000000,2021-03-03
...,...,...,...,...,...
506,2021-04-10,strategizing,0.000000,0.000000,2021-04-10
545,2021-04-10,type,0.000000,0.000000,2021-04-10
584,2021-04-10,wake up,0.000000,0.000000,2021-04-10
311,2021-04-10,nodejs,0.000000,0.000000,2021-04-10


In [33]:
filtered_rolling_avgs = tidy_rolling_avgs[(tidy_rolling_avgs["day"].dt.date >= datetime.date(2021,3,3)) & (
    tidy_rolling_avgs["day"].dt.date <= datetime.date(2021,4,3))]
selection2 = alt.selection_multi(fields=[ACTIVITY_OR_CATEGORY])

c2 = alt.Chart(filtered_rolling_avgs).mark_bar().encode(
    x=alt.X('monthdate(day):O', axis=alt.Axis(title='Date')),
    y=alt.Y('mean', axis=alt.Axis(title=f"{1}-Day Rolling Average")),
    order=alt.Order('mean', sort="descending"),
    color=alt.Color(ACTIVITY_OR_CATEGORY, legend=None),
    opacity=alt.condition(selection2, alt.value(1), alt.value(.2)),
    tooltip=[
        f"{ACTIVITY_OR_CATEGORY}:N",
        alt.Tooltip('mean:Q', format='.2f', title='mean'),
        alt.Tooltip('total:Q', format='.2f', title='total'),
        alt.Tooltip('monthdate(lookback):O', title='lookback'),
        alt.Tooltip('monthdate(day):O', title='day')
        ]
).transform_filter(alt.FieldOneOfPredicate(field=ACTIVITY_OR_CATEGORY, oneOf=SELECTED_ACTIVITIES)).add_selection(
    selection2
)

NameError: name 'SELECTED_ACTIVITIES' is not defined